## <center> Response Surface Methode <br> am Beispiel Bieranalytik <center>
#    <center> von Eike Ahlers <center>

# <center>Gliederung<center>

- Grundlagen
- Vorgehensweise
- Auswertung
- Übung


# <center>Grundlagen<center>

# <center>Was ist die Response Surface Methode<center>

- Sammlung statischer und mathematischer Methoden zur Optimierung von Systemen
- Die Auswirkung y = g(x1,...,xn) +ε, wobei n = Anzahl der Einflussfaktoren, 
    <br> ε = unbekannter Fehler



# <center>Ziele der RSM<center>

- Prozessoptimierung
- beste Kombination von Variablen mit dem größten Response
- möglichst wenige Experimente

Idee der RSM

- Aus Experimente weites vorgehen erschließen
- Resultat der Modelbildung ist eine Gleichung


Nachteil One-Faktor-At-A-Time

- RSM = Änderung mehrerer Faktoren pro Experiment
- COST = Änderungng eines Faktors pro Experiment

<center><video data-autoplay src="test.mp4"></video><center>

# <center>Vorgehensweise<center>
<center><img src="RSM1.png"  height="400" width="800"><center>


Der optimale Betriebsbereich y befindet sich im roten Bereich. Es wird versucht den Prozess immer näher in deisen Bereich zu bewegen.
Zunächst befindet sich der Prozess weit weg und wird mittels faktorielle Experimente in Richtung des optimane Responses gelenkt.
Befindet sich der Prozess in der Nähe vom Optimum werden genauere Methoden angewendet. 
So wird der Rahmen der Untersuchungen immer weiter in Richtung des Optimums geleitet.

# <center>Beispiel Screening (Popcorn)<center>


-2 Faktoren: (Ölsorte und Zeit)

<center><video data-autoplay src="SPlot.mp4"></video><center>

-> Nur Faktor A ist relevant. B kann vernachlässigt werden
Folie runter für Rechenbeispiel

In [ ]:
library(pid)
Zeit    <- c(-1, +1, -1, +1) # Code Einheiten für A
Ölsorte    <- c(-1, -1, +1, +1) # Code Einheiten für B
Response    <- c(74, 106, 76, 104) # Auswirkung: unten links,unten rechts,oben links,oben rechts

model   <- lm(Response ~ Zeit * Ölsorte ) # Haupteffekte mit Interaktionen

Konturdiagramm = contourPlot(model)
ParetoPlot = paretoPlot(model)

Konturplot

<center><video data-autoplay src="CPlot.mp4"></video><center>

## <center>Teil 1: Anweden des first order models an einer zufälligen Stelle<center>
#    <center> Zunächst laden wir die Daten<center>

In [ ]:
library(rsm,readr)
data <- read.csv("TempZeit.csv", sep=" ") #Laden der Datei mit den Experimenten und Ergebnissen
names(data)[1] <- "x1" #nicht beachten
data # zeige die Daten



In [ ]:
RSM <- as.coded.data(data, 
              x1 ~ (Time-35)/5,
              x2 ~ (Temp-155)/5) # Daten in Coded Units umwandelt. Dies braucht R zum berechnen
model <- rsm(Y ~ FO(x1, x2), data = RSM) # FO = first order model # TWI Interaktionen
summary(model) # Wiedergabge der Ergebnisse
contour(model, ~ x1+x2, 
        image = TRUE,
        xaxp=c(30,40,5),
        yaxp=c(150,160,2),
        xlabs=c("Zeit (min)", "Temperatur (°C)")
        )

## <center>Teil 2: Anweden des first order models am neuen Hochpunkt<center>
#    <center> Zunächst laden wir die Daten<center>

In [ ]:
library(rsm,readr)
data <- read.csv("TempZeit2.csv", sep=" ") #Laden der Datei mit den Experimenten und Ergebnissen
names(data)[1] <- "x1" #nicht beachten
data # zeige die Daten

## <center>Nachdem die Daten geladen sind können wir das Modell durch R berechnen lassen<center>

In [ ]:
RSM <- as.coded.data(data, 
              x1 ~ (Time-85)/5,
              x2 ~ (Temp-175)/5) # Daten in Coded Units umwandelt. Dies braucht R zum berechnen
model <- rsm(Y ~ FO(x1, x2)+TWI(x1,x2), data = RSM) # FO = first order model # TWI Interaktionen
summary(model) # Wiedergabge der Ergebnisse
contour(model, ~ x1+x2, 
        image = TRUE,
        xaxp=c(30,40,5),
        yaxp=c(150,160,2),
        xlabs=c("Zeit (min)", "Temperatur (°C)")
        )

## <center>Teil 3 Testen des Central Composite Designs<center>
# <center>Zunächst laden wir die Daten<center>

In [ ]:
library(rsm,readr)
data <- read.csv("CenterCD.csv", sep=" ") #Laden der Datei mit den Experimenten und Ergebnissen
names(data)[1] <- "x1" #nicht beachten
data # zeige die Daten

## <center>Nachdem die Daten geladen sind können wir das Central Composite Design durch R berechnen lassen.<center>
    
## <center>Hierbei wird der Befehl PQ(x1,x2) verwendet. Es steht für den quadratischen Anteil<center>

In [ ]:
RSM <- as.coded.data(data, 
              x1 ~ (Time-85)/5,
              x2 ~ (Temp-175)/5) # Daten in Coded Units umwandelt. Dies braucht R zum berechnen
model <- rsm(Y ~ FO(x1, x2)+PQ(x1, x2), data = RSM) #SO = second order model. Dies beinhaltet FO, TWI und PQ = Quadratischer Anteil
summary(model) # Wiedergabge der Ergebnisse
contour(model, ~ x1+x2, 
        image = TRUE,
        xaxp=c(30,40,5),
        yaxp=c(150,160,2),
        xlabs=c("Zeit (min)", "Temperatur (°C)")
        )

#Darstellung als Response Surface
persp(model, x1~x2, col = terrain.colors(50), contours = "colors",
      zlab = "Yield (%)",
      xlabs=c("Zeite (min)", "Temperatur (°C)")
      )

## <center>Nun wissen wir, das Maximum liegt bei 86.80728 Minuten und 176.28629 °C <center>
# <center>Aber wie groß ist der Response (Auswirkung) an dieser Stelle?<center>
# <center>Wir benutzen den Befehl "predict"<center>

In [ ]:
max <- data.frame(x1 = 0.361, x2 = 0.257) # Die x,x2 (coded Units) Werte für die Prognose kommen aus der Berechnung oben
predict(model, max)

## <center>Da Modell prognostiziert uns ein maximalen Response von 80,2% <center>

In [ ]:
library(rsm)

# creates design with central points:
dsg <- cube(2, n0 = 5, randomize = FALSE)
colnames(dsg)[4]  <- "name"
#dsg$Zeit <- 5*x1 +85

print(dsg)


In [ ]:
P <- c(  0,  -1,  +1,  -1,  +1)
T <- c(  0,  -1,  -1,  +1,  +1)
y <- c(407, 193, 468, 310, 571) #Auswirkung: nullpunkt,unten links,unten rechts,oben links,oben rechts
mod.base.1 <- lm(y ~ P*T)
summary(mod.base.1)
r=contourPlot(mod.base.1, "P", "T")

In [ ]:
P <- c(  0,  -1,  +1,  -1,  +1)
T <- c(  0,  -1,  -1,  +1,  +1)
y <- c(679.5, 571, 688, 630, 733) #Auswirkung: nullpunkt,unten links,unten rechts,oben links,oben rechts
mod.base.1 <- lm(y ~ P*T)
summary(mod.base.1)
r=contourPlot(mod.base.1, "P", "T")